<a href="https://colab.research.google.com/github/spatialthoughts/projects/blob/master/imd/imd_annual_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert IMD Gridded Data from Binary format to GeoTiff

We will use the open-source [imdlib](https://pypi.org/project/imdlib/) library to read the binary files and convert the results to GeoTiff.

This notebook is easiest run on Colab by clicking the 'Open in Colab' button and then running the cells below.

In [1]:
%%capture
!pip install imdlib
!pip install rioxarray

In [2]:
import os
import imdlib as imd
import rioxarray as xr

In [3]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

## Download Data


In [ ]:
year = 2021
variable = 'rain' # other options are ('tmin'/ 'tmax')
data = imd.get_data(variable, 1971, 2004, fn_format='yearwise', file_dir=data_folder)

## Read Data

In [16]:
data = imd.open_data(variable, 1971, 2004,'yearwise', data_folder)
ds = data.get_xarray()
ds = ds.where(ds['rain'] != -999.)
ds

<xarray.Dataset>
Dimensions:  (time: 12419, lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 6.5 6.75 7.0 7.25 7.5 ... 37.5 37.75 38.0 38.25 38.5
  * lon      (lon) float64 66.5 66.75 67.0 67.25 67.5 ... 99.25 99.5 99.75 100.0
  * time     (time) datetime64[ns] 1971-01-01 1971-01-02 ... 2004-12-31
Data variables:
    rain     (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2022-09-22 09:02:55.470752 Python
    references:   
    comment:      
    crs:          epsg:4326

## Aggregate Data

Compute average annual precipitation. Remember we must first sum daily data to yearly and then calculate mean across years.

In [29]:
annual = ds.groupby('time.year').sum('time')
annual

<xarray.Dataset>
Dimensions:  (year: 34, lat: 129, lon: 135)
Coordinates:
  * lat      (lat) float64 6.5 6.75 7.0 7.25 7.5 ... 37.5 37.75 38.0 38.25 38.5
  * lon      (lon) float64 66.5 66.75 67.0 67.25 67.5 ... 99.25 99.5 99.75 100.0
  * year     (year) int64 1971 1972 1973 1974 1975 ... 2000 2001 2002 2003 2004
Data variables:
    rain     (year, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [26]:
average = annual.mean('year')
average

## Save Data as GeoTIFF

Run the cell. Once finished, in Colab, expand the files panel on the left, browse to the `output` folder and download the data.

In [28]:
average = average.rio.set_crs('EPSG:4326')

average = average.rio.set_spatial_dims('lon', 'lat')

output_file = 'annual_average.tif'
output_path = os.path.join(output_folder, output_file)
average.rio.to_raster(output_path)